# Aula 10: Thrust para programação paralela em GPU

### Nome: Luca Mizrahi

In [ ]:
!nvidia-smi

Fri Sep 13 11:26:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
%%writefile exemplo1.cu
#include <thrust/device_vector.h> // Inclui a biblioteca para vetores na GPU (device)
#include <thrust/host_vector.h>   // Inclui a biblioteca para vetores na CPU (host)
#include <iostream>               // Biblioteca padrão para entrada e saída de dados

int main() {
    // Cria um vetor na CPU (host) com 5 elementos, todos inicializados com 0
    thrust::host_vector<double> host(5, 0);
    host[4] = 35; // Altera o último elemento do vetor para 35

    /* Os dados do vetor 'host' são copiados para a GPU,
       criando um vetor equivalente na GPU (device) */
    thrust::device_vector<double> dev(host);

    /* Altera o vetor na CPU, mas não afeta o vetor na GPU,
       pois a cópia já foi feita anteriormente */
    host[2] = 12; // Altera o terceiro elemento do vetor na CPU para 12

    // Exibe os elementos do vetor na CPU
    printf("Host vector: ");
    for (auto i = host.begin(); i != host.end(); i++) {
        std::cout << *i << " "; // Acesso rápido aos elementos na CPU
    }
    printf("\n");

    // Exibe os elementos do vetor na GPU
    printf("Device vector: ");
    for (auto i = dev.begin(); i != dev.end(); i++) {
        std::cout << *i << " "; // Acesso aos elementos na GPU é lento, pois os dados estão na GPU
    }
    printf("\n");
}

Overwriting exemplo1.cu


In [11]:
!nvcc -arch=sm_75 -std=c++14 exemplo1.cu -o exemplo1

In [12]:
!./exemplo1

Host vector: 0 0 12 0 35 
Device vector: 0 0 0 0 35 


In [13]:
%%writefile exemplo2.cu
#include <thrust/device_vector.h>              // Inclui a biblioteca para vetores na GPU
#include <thrust/host_vector.h>                // Inclui a biblioteca para vetores na CPU
#include <thrust/sequence.h>                   // Inclui a função para preencher vetores com uma sequência de números
#include <thrust/functional.h>                 // Inclui operações matemáticas padrão, como soma e multiplicação
#include <thrust/transform.h>                  // Inclui a função para transformar vetores com operações ponto a ponto
#include <thrust/iterator/constant_iterator.h> // Inclui o iterador constante necessário
#include <iostream>

int main() {
    // Cria um vetor na GPU (device_vector) com 10 elementos, todos inicializados com 0
    thrust::device_vector<double> V1(10, 0);
    // Preenche V1 com uma sequência de números: {0, 1, 2, ..., 9}
    thrust::sequence(V1.begin(), V1.end());

    // Cria um vetor na GPU com 5 elementos, todos inicializados com 0
    thrust::device_vector<double> V2(5, 0);
    // Preenche os dois primeiros elementos de V2 com 5.5: {5.5, 5.5, 0, 0, 0}
    thrust::fill(V2.begin(), V2.begin() + 2, 5.5);
    // Preenche os elementos restantes de V2 com 10: {5.5, 5.5, 10, 10, 10}
    thrust::fill(V2.begin() + 2, V2.end(), 10);

    // Cria dois vetores na GPU com 10 elementos, para armazenar resultados das operações
    thrust::device_vector<double> V3(10); // Vetor para armazenar o resultado da soma
    thrust::device_vector<double> V4(10); // Vetor para armazenar o resultado da multiplicação

    // Aplica a operação de soma elemento por elemento entre V1 e V2 e armazena o resultado em V3
    // Como V2 tem menos elementos, o restante de V1 é somado com zeros (elemento padrão).
    // Resultado: V3 = {0+5.5, 1+5.5, 2+10, 3+10, 4+10, 5+0, ..., 9+0}
    thrust::transform(V1.begin(), V1.end(), V2.begin(), V3.begin(), thrust::plus<double>());

    // Aplica multiplicação de V1 com o valor constante 0.5
    // e armazenaria o resultado em V4.
    thrust::transform(V1.begin(), V1.end(), thrust::constant_iterator<double>(0.5), V4.begin(), thrust::multiplies<double>());

    // Imprime os elementos de V1
    printf("V1: ");
    for (thrust::device_vector<double>::iterator i = V1.begin(); i != V1.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V1
    }
    printf("\n");

    // Imprime os elementos de V2
    printf("V2: ");
    for (thrust::device_vector<double>::iterator i = V2.begin(); i != V2.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V2
    }
    printf("\n");

    // Imprime os elementos de V3
    printf("V3: ");
    for (thrust::device_vector<double>::iterator i = V3.begin(); i != V3.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V3
    }
    printf("\n");

    // Imprime o vetor V4, que foi criado mas não utilizado.
     printf("V4: ");
    for (thrust::device_vector<double>::iterator i = V4.begin(); i != V4.end(); i++) {
         std::cout << *i << " ";
    }
    printf("\n");

    return 0;
}

Overwriting exemplo2.cu


In [14]:
!nvcc -arch=sm_75 -std=c++14 exemplo2.cu -o exemplo2

In [15]:
!./exemplo2

V1: 0 1 2 3 4 5 6 7 8 9 
V2: 5.5 5.5 10 10 10 
V3: 5.5 6.5 12 13 14 5 6 7 8 9 
V4: 0 0.5 1 1.5 2 2.5 3 3.5 4 4.5 


#### *Atividade 1*

In [16]:
%%writefile stocks.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    int n = 2518;                   // Define o tamanho do vetor (2518 elementos)
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) com n elementos, todos inicializados com 0
    thrust::host_vector<double> host(n, 0);

    // Loop para ler n valores da entrada padrão e armazená-los no vetor host
    for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    return 0;
}

Overwriting stocks.cu


In [17]:
!nvcc -arch=sm_70 -std=c++14 stocks.cu -o stocks

In [18]:
!./stocks < stocks-google.txt

Tempo de LEITURA (em segundos)  0.000513716
Tempo de CÓPIA (em segundos)  0.221591


#### *Atividade 2*

In [31]:
%%writefile stocks_reduce.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <thrust/reduce.h>          // Inclui a função de redução (soma)
#include <thrust/extrema.h>         // Inclui as funções min_element e max_element
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código

using namespace std;

int main() {
    int n = 389;  // Número de linhas do arquivo
    int last_year_days = 365;  // Último ano
    double value = 0.0;

    // Criação do host_vector para armazenar os dados na CPU
    thrust::host_vector<double> host(n, 0);

    // Lê os dados de entrada
    for (int i = 0; i < n; i++) {
        cin >> value;
        host[i] = value;
    }

    // Transfere os dados para a GPU (device_vector)
    thrust::device_vector<double> dev(host);

    // 1. Calcula o preço médio dos últimos 10 anos
    double sum_total = thrust::reduce(dev.begin(), dev.end(), 0.0, thrust::plus<double>());
    double avg_total = sum_total / n;
    cout << "Preço médio do arquivo todo: " << avg_total << endl;

    // 2. Calcula o preço médio dos últimos 365 dias
    double sum_last_year = thrust::reduce(dev.end() - last_year_days, dev.end(), 0.0, thrust::plus<double>());
    double avg_last_year = sum_last_year / last_year_days;
    cout << "Preço médio nos últimos 365 dias: " << avg_last_year << endl;

    // 3. Encontra o maior e menor preço nos últimos 10 anos
    auto max_total = thrust::max_element(dev.begin(), dev.end());
    auto min_total = thrust::min_element(dev.begin(), dev.end());
    cout << "Maior preço do arquivo todo: " << *max_total << endl;
    cout << "Menor preço do arquivo todo: " << *min_total << endl;

    // 4. Encontra o maior e menor preço nos últimos 365 dias
    auto max_last_year = thrust::max_element(dev.end() - last_year_days, dev.end());
    auto min_last_year = thrust::min_element(dev.end() - last_year_days, dev.end());
    cout << "Maior preço nos últimos 365 dias: " << *max_last_year << endl;
    cout << "Menor preço nos últimos 365 dias: " << *min_last_year << endl;

    return 0;
}

Overwriting stocks_reduce.cu


In [32]:
!nvcc -arch=sm_75 -std=c++14 stocks_reduce.cu -o stocks_reduce

In [33]:
!./stocks_reduce < stocks-google.txt

Preço médio do arquivo todo: 1580.08
Preço médio nos últimos 365 dias: 1615.81
Maior preço do arquivo todo: 2200
Menor preço do arquivo todo: 1015.24
Maior preço nos últimos 365 dias: 2200
Menor preço nos últimos 365 dias: 1060.5


In [27]:
import pandas as pd

df = pd.read_csv("stocks-google.txt", header=None)

df.describe()

,0
count,389.000000
mean,1580.077095
std,352.293584
min,1015.240000
25%,1268.500000
50%,1571.750000
75%,1884.750000
max,2200.000000


#### *Atividade 3*

In [39]:
%%writefile stocks_diff.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <thrust/transform.h>       // Inclui a função transform para operações ponto a ponto
#include <thrust/reduce.h>          // Inclui a função reduce para somar os elementos do vetor
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <sstream>                  // Para manipular a leitura das linhas do arquivo
#include <fstream>                  // Biblioteca para manipulação de arquivos

using namespace std;

int main() {
    int n = 3520;  // Aproximadamente 352 dias de negociação por ano, durante 10 anos

    // Criação dos host_vectors para armazenar os preços na CPU
    thrust::host_vector<double> aapl_host(n, 0);
    thrust::host_vector<double> msft_host(n, 0);

    // Abrir o arquivo stocks2.txt
    std::ifstream file("stocks2.txt");
    if (!file) {
        std::cerr << "Erro ao abrir o arquivo stocks2.txt!" << std::endl;
        return 1;
    }

    // Lê os dados do arquivo e armazena nos host_vectors
    std::string line;
    int index = 0;
    while (std::getline(file, line)) {
        std::stringstream ss(line);
        std::string aapl_str, msft_str;

        // Separa os valores de AAPL e MSFT
        std::getline(ss, aapl_str, ',');
        std::getline(ss, msft_str, ',');

        // Converte os valores para double e armazena nos vetores
        aapl_host[index] = std::stod(aapl_str);
        msft_host[index] = std::stod(msft_str);
        index++;
    }
    file.close();

    // Transfere os dados para a GPU (device_vector)
    thrust::device_vector<double> aapl_dev = aapl_host;
    thrust::device_vector<double> msft_dev = msft_host;

    // Cria um device_vector para armazenar as diferenças ponto a ponto
    thrust::device_vector<double> diff_dev(n);

    // Calcula a diferença ponto a ponto entre os preços de AAPL e MSFT
    thrust::transform(aapl_dev.begin(), aapl_dev.end(), msft_dev.begin(), diff_dev.begin(), thrust::minus<double>());

    // Calcula a média das diferenças
    double sum_diff = thrust::reduce(diff_dev.begin(), diff_dev.end(), 0.0, thrust::plus<double>());
    double avg_diff = sum_diff / n;

    // Exibe a média das diferenças
    cout << "Média das diferenças (AAPL - MSFT) nos últimos 10 anos: " << avg_diff << endl;

    // Verifica qual ação teve o preço médio maior
    if (avg_diff > 0) {
        cout << "A Apple (AAPL) teve, em média, um preço maior." << endl;
    } else if (avg_diff < 0) {
        cout << "A Microsoft (MSFT) teve, em média, um preço maior." << endl;
    } else {
        cout << "As duas empresas tiveram preços médios iguais." << endl;
    }

    return 0;
}

Overwriting stocks_diff.cu


In [40]:
!nvcc -arch=sm_75 -std=c++14 stocks_diff.cu -o stocks_diff

In [41]:
!./stocks_diff

Média das diferenças (AAPL - MSFT) nos últimos 10 anos: -265.232
A Microsoft (MSFT) teve, em média, um preço maior.
